In [1]:
!pip install shap lime ollama python-dotenv
#!pip install "numpy==1.26.4" --force-reinstall
#!pip install "shap<0.50" lime ollama python-dotenv



In [2]:
import os
import sys
import importlib

# Dacă rulezi notebook-ul din folderul demo/, cwd = .../xai-nlg-framework/demo
# Deci root-ul proiectului e un nivel mai sus:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import src.pipeline
importlib.reload(src.pipeline)

from config.settings import FrameworkConfig
from src.pipeline import XAINLGPipeline
from src.nlg.few_shot_generator import FewShotGenerator
from src.nlg.ollama_client import ollama_llm_call


dataset + model

In [3]:
# 1. Dataset
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = list(data.feature_names)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Config + pipeline
config = FrameworkConfig(verbose = True)
config.explainer.top_k_features = 5 
pipeline = XAINLGPipeline(
    model,
    X_train,
    feature_names,
    config=config,
)


len(X_train), len(X_test), len(feature_names)


(455, 114, 30)

pipeline + explicație SHAP

In [4]:
# alegem o instanță de explicat
instance = X_test[0]
raw_pred = int(model.predict(instance.reshape(1, -1))[0])
print("Raw prediction:", raw_pred)

# generăm explicația SHAP (fără NLG intern)
result = pipeline.explain_instance(
    instance,
    method="shap",
    generate_text=False,   # NLG îl facem noi în notebook
)

result.keys()


INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30


Raw prediction: 1


dict_keys(['instance', 'method', 'explanation', 'base_value', 'prediction', 'ranked_features', 'statements'])

NLG cu Few-Shot + Ollama

In [5]:
# === Layer 3: NLG cu Ollama (Few-Shot) ===

# generator NLG care folosește llama3:latest din Ollama
nlg = FewShotGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# Top-k features din explicația SHAP
top_k = result["ranked_features"][: config.explainer.top_k_features]
top_k



[('worst area', 0.06724261426271144),
 ('worst concave points', 0.05740842121890832),
 ('mean concave points', 0.04227830540602773),
 ('worst radius', 0.03956915260512325),
 ('worst perimeter', 0.03026387093604719)]

In [6]:

directions = []
for _, v in top_k:
    if v > 0:
        directions.append("supports")
    elif v < 0:
        directions.append("contradicts")
    else:
        directions.append("neutral")

raw_pred = int(result["prediction"])

pred_expert_en = "malignant tumor" if raw_pred == 0 else "benign tumor"
pred_layman_en = "high risk of breast cancer" if raw_pred == 0 else "low risk of breast cancer"

base_context = {
    "features": [f for f, _ in top_k],
    "values": [float(v) for _, v in top_k],
    "directions": directions,
    "method": result.get("method", "shap"),
}


## generăm efectiv textul cu Llama3 prin Ollama

In [7]:
from src.nlg.few_shot_generator import FewShotGenerator
from src.nlg.ollama_client import ollama_llm_call

# puțin mai „cuminte” pentru format fix
config.nlg.temperature = 0.2  

nlg = FewShotGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# 1) context pentru EXPERT
context_expert = {
    **base_context,
    "prediction": pred_expert_en,   # ex: "malignant tumor" / "benign tumor"
    "audience": "expert",
}

# 2) context pentru LAYMAN
context_layman = {
    **base_context,
    "prediction": pred_layman_en,   # ex: "high risk of breast cancer" / "low risk of breast cancer"
    "audience": "layman",
}

text_expert = nlg.generate(context_expert)
text_layman = nlg.generate(context_layman)

print("Explicație tehnică (pentru medic sau cercetător):")
print(text_expert)
print("\nPe scurt, pentru un pacient:")
print(text_layman)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


Explicație tehnică (pentru medic sau cercetător):
The model predicts a benign tumor because several morphological features have small positive contributions to the malignancy score. The worst area, concave points, radius, and perimeter all exhibit low values that support the benign class, with their combined effects dominating the overall attribution. These small but consistent positive contributions outweigh any potential opposing factors, ultimately leading to a prediction of a benign tumor.

Pe scurt, pentru un pacient:
The model suggests a low risk of breast cancer because the lump's size and shape are quite small and regular, which is more typical of harmless findings. The texture measurements also point towards a benign outcome, indicating that this lump does not have the characteristics commonly seen in cancer cases. Overall, these factors together suggest that the lump is likely to be non-cancerous.


## Chain of Thought

In [ ]:
from src.utils import get_top_k_features

# ranked_features = list of (feature_name, value)
top_k = result["ranked_features"][: config.explainer.top_k_features]

directions = []
for _, v in top_k:
    if v > 0:
        directions.append("supports")
    elif v < 0:
        directions.append("contradicts")
    else:
        directions.append("neutral")

raw_pred = int(result["prediction"])

pred_expert_en = "malignant tumor" if raw_pred == 0 else "benign tumor"
pred_layman_en = "high risk of breast cancer" if raw_pred == 0 else "low risk of breast cancer"

base_context = {
    "features": [f for f, _ in top_k],
    "values": [float(v) for _, v in top_k],
    "directions": directions,
    "method": result.get("method", "shap"),
}


In [ ]:
from src.nlg.cot_generator import ChainOfThoughtGenerator
from src.nlg.ollama_client import ollama_llm_call

cot_gen = ChainOfThoughtGenerator(config.nlg, llm_call_fn=ollama_llm_call)

context_cot_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

text_cot_expert = cot_gen.generate(context_cot_expert)

print("=== Chain-of-Thought (technical) ===")
print(text_cot_expert)

context_cot_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

text_cot_layman = cot_gen.generate(context_cot_layman)

print("\n=== Chain-of-Thought (simple) ===")
print(text_cot_layman)



## Self_conistency

In [8]:
from src.nlg.self_consistency_generator import SelfConsistencyGenerator

sc_gen = SelfConsistencyGenerator(
    config.nlg,
    n_chains=3,                    # poți pune 3–5, dar ține cont de latență
    llm_call_fn=ollama_llm_call,
)

context_sc_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

text_sc_expert = sc_gen.generate(context_sc_expert)

print("=== Self-Consistency (aggregated, technical) ===")
print(text_sc_expert)

context_sc_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

text_sc_layman = sc_gen.generate(context_sc_layman)

print("\n=== Self-Consistency (aggregated, simple) ===")
print(text_sc_layman)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


=== Self-Consistency (aggregated, technical) ===
The model predicts a benign tumor based on several key factors. The worst area and concave points of the tumor strongly support this prediction, indicating that its shape is not irregular or suspicious. Additionally, the mean concave points and radius contribute to the conclusion that it is benign, suggesting that the tumor's overall structure and dimensions are consistent with a benign diagnosis.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



=== Self-Consistency (aggregated, simple) ===
The model's prediction of a low risk of breast cancer is supported by several favorable features in the input data. The presence of worst area, worst concave points, mean concave points, worst radius, and worst perimeter all contribute to this outcome, indicating that the tumor has a relatively benign shape and size. Specifically, the less concerning worst area and concave points suggest a less aggressive growth pattern, while the mean concave points and radius indicate a more symmetrical and compact growth pattern, further supporting a low risk of breast cancer diagnosis.


In [ ]:
for tech in ["few_shot", "cot", "self_consistency"]:
    print("\n==============================")
    print(f"TECHNIQUE = {tech}")
    print("==============================")

    res = pipeline.explain_instance(
        instance,
        method="shap",
        generate_text=True,
        technique=tech,        # 👈 AICI testăm ramura
        audience="expert",     # pentru few_shot; la celelalte e ignorat
    )

    print("Reported technique:", res.get("nlg_technique"))
    print("Reported generator:", res.get("nlg_generator"))
    print("\nGenerated text:\n")
    print(res.get("generated_text", "")[:800])  # primele 800 caractere
    print("\nValidation:", res.get("validation"))
